In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
#import cv2 
import IPython
from glob import glob
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import tqdm
#import seaborn as sns
import albumentations as A
import torch
import torch.nn as nn
import timm
from torcheval.metrics import R2Score
import wandb
import torchvision


/home/prajwal/anaconda3/envs/pytorch_gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:

BASE_DIR = os.path.join(os.getcwd() , 'data')
train_df = pd.read_csv(BASE_DIR  +  '/train.csv')
TRAIN_VAL_SPLIT_SIZE = 0.2
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 128
TEST_BATCH_SIZE  = 10
LEARNING_RATE = 1e-5
EPOCHS = 10

Normalize_transform_type = "log_transform"
RANDOM_NUMBER = 42
torch.manual_seed(RANDOM_NUMBER)

In [3]:
#wandb.login()

# wandb.init(project="cs680v2_normalized",
#     config={
#        "learning_rate": LEARNING_RATE,
#         "epochs": EPOCHS,
#         "batch_size" : TRAIN_BATCH_SIZE,
#     }
# )

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIRECTORY = os.path.join(os.getcwd(),"data")


torch.cuda.empty_cache()

In [5]:
DEVICE

device(type='cuda')

In [6]:
def create_val_train_dataset(BASE_DIR,train_split_size):
    df = pd.read_csv(BASE_DIR  +  '/train.csv')
    df["image_path"] = BASE_DIR + "/train_images/" + df["id"].astype(str) + ".jpeg"   # add images in the training dataframe
    train_df , val_df = train_test_split(df,test_size=train_split_size,shuffle=True)
    train_df.reset_index(drop=True , inplace  = True)
    val_df.reset_index(drop = True , inplace = True)
    return  train_df , val_df


def create_test_dataset(BASE_DIR):
    df = pd.read_csv(BASE_DIR  +  '/test.csv')
    df["image_path"] = BASE_DIR + "/test_images/" + df["id"].astype(str) + ".jpeg"   # add images in the training dataframe
    return df

train_df , val_df = create_val_train_dataset(BASE_DIR,TRAIN_VAL_SPLIT_SIZE)
test_df = create_test_dataset(BASE_DIR)

In [7]:
class NormalizeOutput():
    def __init__(self,type_transform ):
        self.transform = type_transform
        self.mean = 0
        self.std = 0
        self.mean_tensor = 0
        self.std_tensor = 0
        
        pass
    def normalize(self,df):
        if self.transform == "log_transform":
            mean = np.log10(df).mean()
            std = np.log10(df).std()
            self.mean = mean
            self.std = std
            self.mean_tensor = torch.Tensor(self.mean.values).to(DEVICE)
            self.std_tensor = torch.Tensor(self.std.values).to(DEVICE)
            return (np.log10(df) - self.mean)/self.std
    def denormalize(self,df):
        if self.mean is None or self.std is None :
            raise Exception("mean and/std is not defined ")
        if self.normalize == "log_transform":
            df_denormalize =10**((df*self.std) + self.mean )
            return df_denormalize
    def denormalize_tensor(self,batch) :
        if self.mean_tensor is None or self.std_tensor is None :
            raise Exception("mean and/std is not defined ")
        if self.transform == "log_transform":
            df_denormalize =10**((batch*self.std_tensor) + self.mean_tensor )
            return df_denormalize
            

In [8]:
#img = plt.imread(val_df["image_path"].iloc[0])
# plt.imshow(img)
# img.shape
#train_df.columns

In [9]:
# to avoid overfitting : 1) try diffferent transformations 2) batch norm 3)  dropout  4)

In [10]:
class data_loader(Dataset ):
    def __init__(self,df , is_val = False,normalizedOutput=None ):
        self.df = df.copy()
        if normalizedOutput == None:
            pass
        else :
            if not is_val :
                self.df[['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean' ]] = normalizedOutput.normalize(self.df[['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean' ]])
        print("outside pass")
        self.transform =torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                                 #torchvision.transforms.Resize((384,384)),
                                  torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                    torchvision.transforms.RandomVerticalFlip(p=0.5),
                                    torchvision.transforms.RandomHorizontalFlip(p=0.5),
                                    torchvision.transforms.RandomChoice([
                                        torchvision.transforms.Lambda(lambda x : x + np.sqrt(0.1)*torch.randn_like(x)),
                                        torchvision.transforms.Lambda(lambda x : x + 0.1*torch.randn_like(x)),
                                        torchvision.transforms.Lambda(lambda x : x + torch.randn_like(x))])
                             ])
    
    def __len__(self):
        return len(self.df)
    
    
    
    # def collate_fn(self,data):
    #     print("hello")
    #     images =[(i["image"]) for i in data]
    #     print(images) 
    #     traits = np.array ( [i["traits"] for i in data] )
    #     #images = [self.resize_image(j) for i in data for j in i["images"]]
        
    #     #images = torch.permute(torch.tensor(images,dtype=torch.float),(0,3,1,2)) # Batch x channels x height x width
    #     #images = self.transform(images)
    #     #images = torch.permute(images , (0,3,1,2))
    #     traits = torch.tensor(traits,dtype=torch.float)
    #     return images , traits
    
    
    def __getitem__(self , index):
        row = self.df.iloc[index]
        image = plt.imread(row["image_path"])
        image = np.copy(image)
        traits = row[['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean' ]].values.astype(np.float64)
        if self.transform :
            image = self.transform(image)
            #image = torch.permute(image  )
        traits = torch.tensor(traits,dtype=torch.float)
        return  image , traits
normalize_function = NormalizeOutput(Normalize_transform_type)
train_dataset = data_loader(train_df, is_val = False,normalizedOutput=normalize_function)
val_dataset  = data_loader(val_df,   is_val = True,normalizedOutput=normalize_function)    

train_dataloader = DataLoader(train_dataset , batch_size = TRAIN_BATCH_SIZE , shuffle=True )
val_dataloader = DataLoader(val_dataset , batch_size =  VAL_BATCH_SIZE , shuffle=False )

outside pass
outside pass


In [11]:
# class data_loader_test(Dataset ):
#     def __init__(self,df  ):
#         self.df = df.copy()
#         self.transform =torchvision.transforms.Compose([
#                                torchvision.transforms.ToTensor(),
#                                  #torchvision.transforms.Resize((384,384)),
#                                   torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
#                                   ])
    
#     def __len__(self):
#         return len(self.df)    
#     def __getitem__(self , index):
#         row = self.df.iloc[index]
#         image = plt.imread(row["image_path"])
#         image = np.copy(image)
#         if self.transform :
#             image = self.transform(image)
#         return  image 
# test_dataset = data_loader_test(test_df)
# test_dataloader = DataLoader(test_dataset,batch_size =TEST_BATCH_SIZE,shuffle=False)

# def predict_test(test_dataset , model):
#     predictions = []
#     for i in test_dataset:
#         model.eval()
#         prediction = model(i.to(DEVICE))
#         prediction  = normalize_function.denormalize_tensor(prediction)
#         predictions.extend(prediction.detach().cpu().numpy())
#     predictions = np.array(predictions)
#     output = np.reshape(predictions,(-1,predictions.shape[-1]))
#     return pd.DataFrame(output , columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112' ])
# output = predict_test(test_dataloader , model)
# output = pd.concat([test_df["id"],output],axis=1 )
# #output.to_csv()

In [12]:
class R2Loss(nn.Module):
    def __init__(self, num_classes=6):
        super(R2Loss, self).__init__()
        # Initialize learnable weights for each class, one weight per class
        # self.class_weights = nn.Parameter(torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0], dtype=torch.float32))
        # Increase weight for X_26_mean

    def forward(self, y_pred, y_true):
        # Calculate residual sum of squares per class
        SS_res = torch.sum((y_true - y_pred) ** 2, dim=0)  # (B, C) -> (C,)
        # Calculate total sum of squares per class
        SS_tot = torch.sum(
            (y_true - torch.mean(y_true, dim=0)) ** 2, dim=0
        )  # (B, C) -> (C,)
        # Calculate R2 loss per class, avoiding division by zero
        r2_loss =SS_res / (SS_tot + 1e-6)  # (C,)
        # Weight the R2 loss by the learnable class weights
        # weighted_r2_loss = self.class_weights * r2_loss
        # Return the mean of the weighted R2 loss
        return torch.mean(r2_loss)

def initialize_model(model_name  ):
    model_ft  = None
    if model_name == "resnet34" :
        """ Resnet34 """
        model = timm.create_model('resnet34' , num_classes=6 )
        return model
    # if model_name == "Swin_Transformer":
    #     model = timm.create_model('swin_large_patch4_window12_384.ms_in22k_ft_in1k' , num_classes = 6 , pretrained=True)
    #     return model 
    if model_name =="convnextv2":
        model = timm.create_model('convnext_tiny.in12k_ft_in1k_384',pretrained=True)
        model.head.fc = nn.Linear(in_features=768 , out_features= 6 , bias=True)
        return model 
    if model_name =="efficientnet_b0":
        model = timm.create_model('cefficientnet_b0',pretrained=True)
        model.trainable = False
        #model.head.fc = nn.Linear(in_features=768 , out_features= 6 , bias=True)
        model_expand = 
        return model 

def get_model_optimizer_lossFunction(model_name,learning_rate):
    global DEVICE
    model = initialize_model(model_name)
    model.to(device = DEVICE)
    loss_function = R2Loss()
    loss_function.to(device=DEVICE)
    optimizer =  torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    #scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1 , patience=3 , verbose = True )  #torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=25 , start_factor= 1 ,end_factor= 1e-4, verbose = True ) #1e-3*1e-4 = 1e-7
    scheduler  = torch.optim.lr_scheduler.CosineAnnealingLR(
        initial_learning_rate = 1e-5,
        decay_steps = total_steps - warmup_steps,
        warmup_target= CFG['MAX_LR'],
        warmup_steps = warmup_steps   )
    # optimizer.to(device = DEVICE)
    return model,optimizer,loss_function , scheduler
class BestModelSaveCallback:
    def __init__(self, save_path):
        self.save_path = save_path
        self.best_accuracy = -1

    def __call__(self, accuracy,model):
        if accuracy > self.best_accuracy:
            self.best_accuracy = accuracy
            model.to(device = "cpu")
            torch.save(model.state_dict(), self.save_path)
            model.to(device=DEVICE)


In [13]:
def train_batch(inputs,model,loss_function,optimizer):
    x,y = inputs
    x = x.to(DEVICE)
    y = y.to(DEVICE)
    model.train()
    prediction = model(x)
    loss = loss_function(prediction,y)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    return loss.detach().cpu().numpy()

@torch.no_grad
def do_prediction(inputs,model, is_val=False):
    global Train_std_tensor , Train_mean_tensor
    x,y = inputs
    x = x.to(DEVICE)
    y = y.to(DEVICE)
    model.eval()
    prediction = model(x)
    if is_val :
        prediction = normalize_function.denormalize_tensor(batch=prediction)
    return prediction.detach().cpu().numpy()

@torch.no_grad()
def validation_loss_batch(inputs,model,loss_function):
    global Train_std_tensor , Train_mean_tensor
    x,y = inputs
    x = x.to(DEVICE)
    y = y.to(DEVICE)
    model.eval()
    prediction = model(x)
    prediction = normalize_function.denormalize_tensor(batch=prediction)
    loss = loss_function(prediction, y)
    return loss.detach().cpu().numpy()

def utils_convert_to_2d_tensors(predictions,targets):
    predictions  = np.array(predictions)
    targets = np.array(targets)
    predictions  = np.reshape(predictions , (-1, predictions.shape[-1]))
    targets  = np.reshape(targets  , (-1 , targets.shape[-1]))
    return torch.Tensor(predictions), torch.Tensor(targets)

def train(trainLoader,valLoader,model,optimizer,loss_function,epochs,best_model_callback):
    #wandb.watch(model,loss_function,log = "all",log_freq=50)
    
    train_epoch_loss , train_epoch_accuracy =[] , []
    val_epoch_loss , val_epoch_accuracy = [],[]
    
    
    for epoch in range(epochs):
        print(f"epoch: {epoch} , lr is { scheduler.get_last_lr()}" )
        train_loss  = [] 
        val_loss ,val_accuracy = [] , []
        
        # batch training loss
        with tqdm.tqdm(total=len(trainLoader)) as trainingLoop:
            for index,batch in enumerate(iter(trainLoader)): 
        
                loss = train_batch(batch,model,loss_function,optimizer)
                train_loss.append(loss)

                trainingLoop.set_description(f"Batch: {index}/{len(trainLoader)}")
                trainingLoop.set_postfix({"training Loss " : loss})
                trainingLoop.update(1)
                ##wandb.log({"Training Loss":loss })
        train_loss  = np.array(train_loss).mean() 
        train_epoch_loss.append(train_loss)
        
        # find training accuracy 
        predictions,targets = [],[]
        for index,batch in enumerate(iter(trainLoader)): 
        
            prediction = do_prediction(batch,model)
            predictions.extend(prediction)
            targets.extend(batch[1].detach().cpu().numpy())
           
        
        predictions = np.array(predictions)
        targets = np.array(targets)
        predictions , targets = utils_convert_to_2d_tensors(predictions , targets)
        # print("predictions=", predictions[0:2])
        # print("targets=",targets[0:2])
        metric = R2Score()
        metric.update(predictions , targets)
        train_epoch_accuracy.append(metric.compute().detach().cpu().numpy())
                
        
        # validation set loss & accuracy  
        predictions,targets = [],[]
        with tqdm.tqdm(total = len(valLoader)) as validationLoop:
            for index,batch in enumerate(iter(valLoader)):
                
                loss = validation_loss_batch(batch,model,loss_function)
                val_loss.append(loss)
                prediction = do_prediction(batch,model,is_val=True)
                predictions.extend(prediction)
                targets.extend(batch[1].detach().cpu().numpy())
                
                validationLoop.set_description(f"Batch: {index}/{len(valLoader)}")
                validationLoop.set_postfix({"Validation loss " : loss}) 
                ##wandb.log({"Vlaidation loss" : loss})
                #wandb.log({"Validation Loss ": val_loss.item()})
                validationLoop.update(1)
        
        
        val_loss  = np.array(val_loss).mean() 
        val_epoch_loss.append(val_loss)
        
        predictions = np.array(predictions)
        targets = np.array(targets)
        # print("predictions-val=", predictions[0:2])
        # print("targets-val=",targets[0:2])
        metric = R2Score()
        predictions , targets = utils_convert_to_2d_tensors(predictions , targets)
        metric.update(predictions , targets)
        val_epoch_accuracy.append(metric.compute().detach().cpu().numpy().item())
        
        scheduler.step(train_loss)
        
        
        best_model_callback(metric.compute().detach().cpu().numpy().item(),model)        # save the best model according to the validation accuracy
        
        
    return train_epoch_loss,val_epoch_loss,train_epoch_accuracy , val_epoch_accuracy






In [14]:
model,optimizer,loss_function,scheduler = get_model_optimizer_lossFunction(model_name = "convnextv2",learning_rate = LEARNING_RATE)
best_model_callback = BestModelSaveCallback(save_path=os.path.join(BASE_DIR,'best_model_transformation_convnextv2.pth'))
train_losses, val_losses , train_accuracies,val_accuracies = train(train_dataloader,val_dataloader,model,optimizer,loss_function,EPOCHS,best_model_callback)


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/convnext_tiny.in12k_ft_in1k_384)
INFO:timm.models._hub:[timm/convnext_tiny.in12k_ft_in1k_384] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
/home/prajwal/anaconda3/envs/pytorch_gpu/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


epoch: 0 , lr is [1e-05]


Batch: 67/68: 100%|██████████| 68/68 [00:14<00:00,  4.63it/s, Validation loss =0.95453155]


epoch: 1 , lr is [1e-05]


Batch: 67/68: 100%|██████████| 68/68 [00:14<00:00,  4.65it/s, Validation loss =1.2981433]


epoch: 2 , lr is [1e-05]


Batch: 67/68: 100%|██████████| 68/68 [00:14<00:00,  4.62it/s, Validation loss =1.0206107] 


epoch: 3 , lr is [1e-05]


Batch: 233/272:  86%|████████▌ | 234/272 [01:31<00:14,  2.56it/s, training Loss =1.0874095]


KeyboardInterrupt: 

In [ ]:

# predictions= torch.tensor([[ 1.2391, 30.3367, 30.3599, 29.7099, 20.0526, 31.2610],
#         [ 1.2679, 30.3845, 30.3134, 29.7083, 20.3929, 31.2463]])
# targets= torch.tensor([[1.1125e+00, 1.4686e+02, 1.9699e+04, 3.4597e+03, 1.5282e+01, 3.9792e+05],
#         [9.7378e-01, 1.5390e+02, 1.9702e+04, 3.4673e+03, 1.4737e+01, 3.9847e+05]])
# metric = R2Score()
# metric.update(predictions , targets)
# metric.compute()





# from sklearn.metrics import r2_score
# predictions= np.array([[ 1.2391, 30.3367, 30.3599, 29.7099, 20.0526, 31.2610],
#         [ 1.2679, 30.3845, 30.3134, 29.7083, 20.3929, 31.2463]])
# targets= np.array([[1.1125e+00, 1.4686e+02, 1.9699e+04, 3.4597e+03, 1.5282e+01, 3.9792e+05],
#         [9.7378e-01, 1.5390e+02, 1.9702e+04, 3.4673e+03, 1.4737e+01, 3.9847e+05]])
# r2_score(y_true = targets , y_pred =predictions)



# targets = torch.tensor([[0.0, 2.0], [1.0, 6.0]])
# predictions = torch.tensor([[0.0, 1.0], [2.0, 5.0]])
# torch.mean(1- (torch.sum((targets - predictions)**2,axis=0)/torch.sum((targets - targets.mean())**2,axis=0)))


# metric = R2Score()
# # input = torch.tensor([[0, 2], [1, 6]])
# # target = torch.tensor([[0, 1], [2, 5]])
# input = torch.tensor([[ 1.2391, 30.3367, 30.3599, 29.7099, 20.0526, 31.2610],
#         [ 1.2679, 30.3845, 30.3134, 29.7083, 20.3929, 31.2463]])
# target= torch.tensor([[1.1125e+00, 1.4686e+02, 1.9699e+04, 3.4597e+03, 1.5282e+01, 3.9792e+05],
#         [9.7378e-01, 1.5390e+02, 1.9702e+04, 3.4673e+03, 1.4737e+01, 3.9847e+05]])
# metric.update(input, target)
# metric.compute()

In [ ]:
def Plot(train_losses,train_accuracies,val_losses,val_accuracies,path):
    plt.plot(train_losses,label = "train loss")
    plt.plot(val_losses,label = "validation loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    #plt.savefig(os.path.join(path,"Loss.png"))
    #wandb.log({"Loss": plt})

    plt.plot(val_accuracies,label = "validation accuracy")
    plt.plot(train_accuracies,label = "train accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend() 
    #plt.savefig(os.path.join(path,"Accuracy.png"))
    #wandb.log({"Accuracy": plt})

In [ ]:
Plot(train_losses,train_accuracies,val_losses,val_accuracies,path = BASE_DIR)